<strong><h1>Battle of Neighbourhoods - Best Location for Italian Restaurant in New York</h1></strong>

<img src ="https://i.ytimg.com/vi/hVTXu8SgVyE/maxresdefault.jpg">

<stong><h3> Table of contents</h3><strong>
    
* [Introduction: Business Problem](#introduction)
* [Data](#data)   

<strong><h2>Introduction: Business Problem</h2><strong> <a name="introduction"></a>

**New York city** has many restaurants that cater to a large number of people with its **wide variety of cuisines** choices. These cuisines come from various ethnic groups who have come to the city from various parts of the world.

However, for any one trying to open a restuarant in the New York city, the choice on the location for the restaurant can be daunting due to the abudance of restaurants around.

In this project, we are trying to find a location for an **Italian Restaurant** in New York city using data science such that there is little or no competition around offering the same cuisine.


<stong><h2>Data</h2><strong> <a name="data"></a>

Based on the problem, we will need the following data for finding the best location for an Italian Restaurant in New York

* Neighboorhood and Boroughs data incuding latitude and longitude data in order to segement them which can be found on 'https://geo.nyu.edu/catalog/nyu_2451_34572'.


* Venue Data for each borough and neighborhood which can be retrived through Foresquare.

 With the above data, we will look at top 10 venues in each of the clusters formed by using K means algorithm and then decide the location based results.